*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using PyTorch Transformers

In [4]:
# Import packages
import os
import sys
import json 
import pandas as pd
import numpy as np
import torch

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.models.transformers.sequence_classification import Processor, SequenceClassifier 
from utils_nlp.common.timer import Timer

In [5]:
# Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs
QUICK_RUN = False

## Introduction 
## [TODO] - Modify for the final model
This notebook fine-tunes and evaluates a pretrained [XLNet](https://arxiv.org/pdf/1906.08237.pdf) model on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.

We use a [sequence classifier](../../utils_nlp/models/bert/sequence_classification.py) that wraps [Hugging Face's PyTorch implementation](https://github.com/huggingface/pytorch-pretrained-BERT) of Google's [BERT](https://github.com/google-research/bert).

In [7]:
TRAIN_DATA_FRACTION = 1
TEST_DATA_FRACTION = 1
NUM_EPOCHS = 3

if QUICK_RUN:
    TRAIN_DATA_FRACTION = 0.01
    TEST_DATA_FRACTION = 0.01
    NUM_EPOCHS = 1

BATCH_SIZE = 32 if torch.cuda.is_available() else 8
DATA_FOLDER = "./temp"
MODEL_CACHE_DIR = "./temp"
TO_LOWER = True
MAX_LEN = 150
BATCH_SIZE_PRED = 512
TRAIN_SIZE = 0.6
LABEL_COL = "genre"
TEXT_COL = "sentence1" 

### [TODO] - Remove Workflow overview

```
model_name = SequenceClassifier.list_supported_models()[0]
num_labels = len(label_encoder.classes_)
processor = Processor(model_name=model_name, cache_dir=temp_dir)
ds = processor.preprocess(text_train, labels_train, max_len=max_len)
classifier = SequenceClassifier(
    model_name=model_name, num_labels=num_labels, cache_dir=temp_dir
)
classifier.fit(ds, device="cuda", num_epochs=1, batch_size=32, num_gpus=None)
```

## Read Dataset

Let's start by loading a subset of the data.  

The following function downloads and extracts the files, if they don't already exists in the data folder.

The MultiNLI dataset is mainly used for natural language inference (NLI) tasks, where the inputs are sentence pairs and the labels are entailment indicators. The sentence pairs are also classified into *genres* that allow for more coverage and better evaluation of NLI models.

In [13]:
df = load_pandas_df(DATA_FOLDER, "train")

## Quick Analysis of Data  

Let's observe our dataset to see what we are working with.  
For our classification task, we use the first sentence only as the text input, and the corresponding genre as the label. We select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

In [15]:
df = df[df["gold_label"] == "neutral"] # Get unique sentences
df[[LABEL_COL, TEXT_COL]].head()

,genre,sentence1
0,government,Conceptually cream skimming has two basic dime...
4,telephone,yeah i tell you what though if you go price so...
6,travel,But a few Christian mosaics survive above the ...
12,slate,It's not that the questions they asked weren't...
13,travel,"Thebes held onto power until the 12th Dynasty,..."


The examples in the dataset, shown below, are grouped into 5 genres

In [16]:
df[LABEL_COL].value_counts()

telephone     27783
government    25784
travel        25783
fiction       25782
slate         25768
Name: genre, dtype: int64

### Train/Test Data Split 
Using SKlearns (model selection library), split the MNLI Dataset into training and testing. Based on the setting of the `QUICK_RUN` flag, we'll be sampling a fraction of the data for our model

In [23]:
df_train, df_test = train_test_split(df, train_size = TRAIN_SIZE, random_state = 0)
df_train = df_train.sample(frac=TRAIN_DATA_FRACTION).reset_index(drop=True)
df_test = df_test.sample(frac=TEST_DATA_FRACTION).reset_index(drop=True)

### Encode the labels into numeric values
Label Encoder makes it easy to encode dataset labels, categorical features into numerical values, between `0` and `n_classes - 1`; where `n` is the number of distinct labels

In [24]:
# encode labels
label_encoder = LabelEncoder()
train_data_labels = label_encoder.fit_transform(df_train[LABEL_COL])
test_data_labels = label_encoder.fit_transform(df_test[LABEL_COL])

# Count unique encoded labels
num_labels = len(np.unique(train_data_labels))

In [29]:
print(num_labels)
print(label_encoder.classes_)
print(train_data_labels)

5
['fiction' 'government' 'slate' 'telephone' 'travel']
[1 0 0 ... 0 0 3]


In [27]:
print(f"Number of unique labels: {num_labels}")
print(f"Number of training examples: {df_train.shape[0]}")
print(f"Number of testing examples: {df_test.shape[0]}")

Number of unique labels: 5
Number of training examples: 78540
Number of testing examples: 52360


In [30]:
# model_name = SequenceClassifier.list_supported_models()[0]
# num_labels = len(label_encoder.classes_)
# processor = Processor(model_name=model_name, cache_dir=temp_dir)
# ds = processor.preprocess(text_train, labels_train, max_len=max_len)
# classifier = SequenceClassifier(
#     model_name=model_name, num_labels=num_labels, cache_dir=temp_dir
# )
# classifier.fit(ds, device="cuda", num_epochs=1, batch_size=32, num_gpus=None)
SequenceClassifier.list_supported_models()

['bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'roberta-base',
 'roberta-large',
 'roberta-large-mnli',
 'xlnet-base-cased',
 'xlnet-large-cased',
 'distilbert-base-uncased',
 'distilbert-base-uncased-distilled-squad']